In [1]:
import dash
import dash_bootstrap_components as dbc
import dash_table
import pandas as pd
import jupyterlab_dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table

import base64
import datetime
import io
import plotly.graph_objs as go
import numpy as np
import pandas as pd

import dash_table

In [2]:
app = dash.Dash("dash_test", external_stylesheets=[dbc.themes.BOOTSTRAP])
viewer = jupyterlab_dash.AppViewer()

In [3]:
np.random.seed(0)
df = pd.DataFrame({
    'Column {}'.format(i): np.random.rand(30) + i*10
    for i in range(6)})

In [4]:
app.layout = html.Div([
    html.Div(
        dcc.Graph(
            id='g1',
            config={'displayModeBar': False}
        ), className='four columns'), 
    html.Div(
        dcc.Graph(
            id='g2',
            config={'displayModeBar': False}
        ), className='four columns'),
    html.Div(
        dcc.Graph(
            id='g3',
            config={'displayModeBar': False}
        ), className='four columns')
], className='row')

In [5]:
def highlight(x, y):
    def callback(*selectedDatas):
        selectedpoints = df.index
        for i, selected_data in enumerate(selectedDatas):
            if selected_data is not None:
                selected_index = [
                    p['customdata'] for p in selected_data['points']
                ]
                if len(selected_index) > 0:
                    selectedpoints = np.intersect1d(
                        selectedpoints, selected_index)
        # set which points are selected with the `selectedpoints` proper
        # and style those points with the `selected` and `unselected`
        # attribute. see
        figure = {
            'data': [
                {
                    'x': df[x],
                'y': df[y],
                'text': df.index,
                'textposition': 'top',
                'selectedpoints': selectedpoints,
                'customdata': df.index,
                'type': 'scatter',
                'mode': 'markers+text',
                'marker': {
                    'color': 'rgba(0, 116, 217, 0.7)',
                    'size': 12,
                    'line': {
                        'color': 'rgb(0, 116, 217)',
                        'width': 0.5
                    }
                },
                'textfont': {
                    'color': 'rgba(30, 30, 30, 1)'
                },
                'unselected': {
                    'marker': {
                        'opacity': 0.3,
                    },
                    'textfont': {
                        # make text transparent when not selected
                        'color': 'rgba(0, 0, 0, 0)'
                    } 
                }
                },
            ],
            'layout': {
                'clickmode': 'event+select',
                'margin': {'l': 15, 'r': 0, 'b': 15, 't': 5},
                'dragmode': 'select',
                'hovermode': 'closest',
                'showlegend': False},
        }
        # Display a rectangle to highlight the previously selected regio
        shape = {
            'type': 'rect',
            'line': {
                'width': 1,
                'dash': 'dot',
                'color': 'darkgrey'
            }
        }
        if selectedDatas[0] and selectedDatas[0]['range']:
            figure['layout']['shapes'] = [dict({
                'x0': selectedDatas[0]['range']['x'][0],
                'x1': selectedDatas[0]['range']['x'][1],
                'y0': selectedDatas[0]['range']['y'][0],
                'y1': selectedDatas[0]['range']['y'][1]
            }, **shape)]
        else:
            figure['layout']['shapes'] = [dict({
                'type': 'rect',
                'x0': np.min(df[x]),
                'x1': np.max(df[x]),
                'y0': np.min(df[y]),
                'y1': np.max(df[y])
            }, **shape)]
        return figure

    return callback

In [6]:
app.callback(
    Output('g1', 'figure'),
    [Input('g1', 'selectedData'),
     Input('g2', 'selectedData'),
     Input('g3', 'selectedData')]
    )(highlight('Column 0', 'Column 1'))

app.callback(
    Output('g2', 'figure'),
    [Input('g2', 'selectedData'),
     Input('g1', 'selectedData'),
     Input('g3', 'selectedData')]
    )(highlight('Column 2', 'Column 3'))

app.callback(
    Output('g3', 'figure'),
    [Input('g3', 'selectedData'),
     Input('g1', 'selectedData'),
     Input('g2', 'selectedData')]
    )(highlight('Column 4', 'Column 5'))

<function __main__.highlight.<locals>.callback(*selectedDatas)>

In [ ]:
# if __name__ == '__main__':
#     app.run_server(debug=True, port=8050)
viewer.show(app)